In [130]:
# 파일 경로 설정
file_path = '/Users/dryoon04/Documents/GitHub/schoolproject/vectorstore/open ai api key.txt'

# 파일 열기 (읽기 모드로 열기)
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽어오기
    file_content = file.read()

In [131]:
import pandas as pd
import chromadb
import os
import openai


In [132]:
df = pd.read_csv('lyrics.csv')

In [133]:
client = chromadb.PersistentClient("chromadb/lyrics.db")
collection = client.get_or_create_collection(name="lyrics",metadata={"hnsw:space":"cosine"})


In [134]:
singers = []
song_titles = []
lyrics = []
ids = []
documents = []

for idx in range(len(df)):
    item = df.iloc[idx]
    singer = item['가수'].strip().lower()  # 가수 정보
    song_title = item['제목'].strip().lower()  # 노래 제목 정보
    lyric = item['가사'].strip().lower()  # 가사 정보

    # id = f"{singer}-{song_title}".replace(' ', '-')  # 고유한 ID 생성
    id = f"{singer}-{song_title}"
    document = f"{singer}:{song_title}:{lyric}"  # 문서 텍스트 생성
    
    
    ids.append(id)
    documents.append(document)

# DB 저장
collection.add(documents=documents,ids=ids)


In [135]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(user_query)
print(vector_res)

퀸카
{'ids': [['fiftyfifty-cupid', '방탄소년단-butter', '방탄소년단-dynamite', '정국-3d(feat.jackharlow)', '(여자)아이들-tomboy']], 'distances': [[0.7944316267967224, 0.7963334321975708, 0.8054043650627136, 0.8069340586662292, 0.8194237351417542]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [["fiftyfifty:cupid:불꺼진 romantic all my life\n내 주위는 온통 lovely day\n내 눈 속에 비친 arrow sign\n(oh why oh why, oh why oh why)\ni’m feeling lonely (lonely)\n그만 힐끗대고 말해줘요\nhold me (hold me)\n다시 crying in my room\n숨기고 싶어 (say what you say\nbut i want it more)\nbut still i want it more, more, more\ni gave a second chance to cupid\n널 믿은 내가 정말 stupid\n보여줄게 숨겨왔던 love, is it real?\ncupid is so dumb\n또 꿈길을 걷는 everyday\n눈 뜨면 다시 또 flew away\nwaiting around is a waste (waste)\n나 솔직히 지금이 편해\n상상만큼 짜릿한 걸까?\nnow i’m so lonely (lonely)\n매일 꿈속에서 연습했죠\nkiss me (kiss me)\n다시 crying in my room\n포기할까봐 (say what you say\nbut i want it more)\nbut still i want it more, more, more\ni gave 

In [146]:

os.environ["OPENAI_API_KEY"] = file_content
openai.api_key = os.getenv("OPENAI_API_KEY")
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(vector_res)
srchres = []
for v in vector_res['documents'][0]:
    item = v.split(':')
    srchres.append({"가수" : item[0].strip(),"제목" : item[1].strip(),"가사" : item[2].strip()})
    
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
               {"role": "system","content": "You are an assistant who can answer questions about music lyricsBased on the following database In order to answer questions, you should only use facts from the databaseIf you feel you don't have enough information to answer a question, you should say, '답할수있는 정보가 없습니다.'."},
            {"role": "user", "content": f"{user_query}"},
            {"role": "assistant", "content": f"{srchres}"},
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    
print(completion["choices"][0]["message"]["content"])


{'ids': [['방탄소년단-butter', '디오(d.o.)-somebody', 'itzy(있지)-cake', '(여자)아이들-퀸카(queencard)', '정국-3d(feat.jackharlow)']], 'distances': [[0.5989470481872559, 0.6057289242744446, 0.6132176164767937, 0.6168206930160522, 0.6344677805900574]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [["방탄소년단:butter:smooth like butter\nlike a criminal undercover\ngon’ pop like trouble\nbreakin’ into your heart like that\ncool shade stunner\nyeah i owe it all to my mother\nhot like summer\nyeah i’m makin’ you sweat like that\nbreak it down\noh when i look in the mirror\ni’ll melt your heart into 2\ni got that superstar glow so \ndo the boogie like\nside step right left to my beat (heartbeat)\nhigh like the moon rock with me baby\nknow that i got that heat\nlet me show you ‘cause talk is cheap\nside step right left to my beat (heartbeat)\nget it, let it roll\nsmooth like butter\npull you in like no other\ndon’t need no usher\nto remind me you g

In [137]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=10)
print(user_query)
print(vector_res)


{'ids': [['charlieputh-that’snothowthisworks(feat.dan+shay)', '지민-likecrazy', "charlieputh-that'shilarious", 'newjeans-asap', 'fiftyfifty-cupid', '정국,방탄소년단-dreamers[musicfromthefifaworldcupqatar2022officialsoundtrack](feat.fifasound)', 'akmu(악뮤)-lovelee', 'newjeans-newjeans', '방탄소년단-butter', 'paulblanco-summer(feat.be’o(비오))']], 'distances': [[0.8533503413200378, 0.8709683418273926, 0.8768119812011719, 0.8802019357681274, 0.8917070031166077, 0.8919647192552597, 0.8931661248207092, 0.8951927423477173, 0.8956776261329651, 0.8999037742614746]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['charlieputh:that’snothowthisworks(feat.dan+shay):thought the day you disappeared\nthat it was over\ndidn\'t even hear you leavin\'\nsaw you with someone and\nthought that it was closure\nbut you still tell me\nthat you need me\nbaby, why, tell me, why\ncan\'t you just make up your mind?\nquit messin\' with 